In [1]:
!pip install torch==2.4.0 transformers==4.45.1 datasets==3.0.1 accelerate==0.34.2 trl==0.11.1 peft==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 12.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of 

In [10]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [11]:
# 허깅페이스 허브에서 데이터셋 로드 ← ❶
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

# system_message 정의 ← ❷
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다."라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

# 원본 데이터의 type별 분포 출력 ← ❸
print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

# train/test 분할 비율 설정 (0.5면 5:5로 분할) ← ❹
test_ratio = 0.8

train_data = []
test_data = []

# type별로 순회하면서 train/test 데이터 분할 ← ❺
for type_name in set(dataset['type']):
    # 현재 type에 해당하는 데이터의 인덱스만 추출
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]
    
    # test_ratio에 따라 test 데이터 개수 계산 
    test_size = int(len(curr_type_data) * test_ratio)
    
    # 현재 type의 데이터를 test_ratio 비율로 분할하여 추가
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

# OpenAI format으로 데이터를 변환하기 위한 함수 ← ❻
def format_data(sample):
    # 검색 결과를 문서1, 문서2... 형태로 포매팅
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])
    
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

# 분할된 데이터를 OpenAI format으로 변환 ← ❼
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 최종 데이터셋 크기 출력 ← ❽
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

# 분할된 데이터의 type별 분포 출력 ← ❾
print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

원본 데이터의 type 분포:
no_answer: 404
mrc_question: 491
paraphrased_question: 196
synthetic_question: 497
mrc_question_with_1_to_4_negative: 296

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
no_answer: 81
mrc_question: 99
paraphrased_question: 40
synthetic_question: 100
mrc_question_with_1_to_4_negative: 60

테스트 데이터의 type 분포:
no_answer: 323
mrc_question: 392
paraphrased_question: 156
synthetic_question: 397
mrc_question_with_1_to_4_negative: 236


In [12]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다."라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: LED(발광다이오드) 조명 등을 만드는 동부그룹 계열사 동부라이텍은 일본 요코하마에 LED 라이트 패널(루미시트) 생산공장을 완공, 본격 양산에 들어갔다고 31일 발표했다. 이 공장은 일본 현지 유통사인 테크타이토와 합작해 세운 공장이다. 루미시트는 얇은 종이판 형태의 LED 조명으로, 이 공장에서는 광고 인테리어용 루미시트 4종을 양산한다.동부라이텍은 2008년 캐나다 토론토에 현지 합작법인 DLC를 세워 북미 고급 매장에서 사용하는 진열대용 루미시트를 생산하고 있다. DLC는 올해 상반기에 약 200억원의 매출을 올렸고, 순이익은 최근 수년간 매년 20%씩 증가하고 있다. 요코하마 공장은 캐나다에서의 성공 모델을 일본으로 옮겨온 것이라는 게 회사 측 설명이다. 동부라이텍은 테크타이토와 합작해 지난해 8월 도쿄에 자본금 1억엔 규모의 합작법인 씨엔디라이텍을 설립한 뒤 현지 공장 가동을 준비해 왔다. 동부라이텍은 테크타이토의 일본 내 유통망을 활용해 일본 루미시트 시장에서 점유율을 늘릴 수 있을 것으로 기대하고 있다.'},
 {'role':

In [13]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [14]:
# 허깅페이스 모델 이름
model_id = "Qwen/Qwen2-7B-Instruct" 

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [15]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다."라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 콘코바르는 결국 죽게 되는데, 그 곡절은 이러하다. 라긴의 왕 메스 게그러의 뇌를 굳힌 겻을 울라의 코날이 전리품으로 가지고 있었는데, 코나크타의 전사 케트 막 마가크가 이를 훔쳐갔다. 그리고 케트는 무릿매로 메스 게그러의 뇌를 던져 콘코바르의 머리를 맞추었고, 메스 게그러의 뇌가 콘코바르의 머리통 깊숙히 박혀 버렸다. 이 일이 일어난 곳은 우르카르(Urchair)의 발러 아흐(Baile Ath), 곧 오늘날의 웨스트미스 주 호르셀리프라고 한다. 콘코바르의 의사들은 이 이물질을 제거할 수 없었고, 상처를 봉합만 한 뒤 왕에게 흥분하지 않으면 생명을 유지할 수 있다고 말했다. 7년이 평화롭게 흘러간 뒤 콘코바르는 그리스도가 죽었다는 소식을 듣게 되어 분노했고, 뇌가 터져 죽었다. 머리가 터진 자리에서 뿜어져나온 피의 세례를 받은 결과 그는 기독교인이 되었고 그 영혼은 천국으로 갔다. 콘코바르의 죽음에 관한 이 기록은 매우 얄팍한 기독교화가 이루어져 있는데, 한편 노르드 신화의 토르가 흐룽그니르와 싸우다 머리에 숫돌이 박힌 이야기와 유사한 점이 있다. 어쩌면 두 이야기는 하나의 기원을 공유하

In [16]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [17]:
args = SFTConfig(
    output_dir="qwen2-7b-rag-ko",        # 저장될 디렉터리와 저장소 ID
    num_train_epochs=3,                  # 학습할 총 에포크 수 
    per_device_train_batch_size=2,       # GPU당 배치 크기
    gradient_accumulation_steps=2,       # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,         # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",           # 최적화기
    logging_steps=10,                    # 로그 기록 주기
    save_strategy="steps",               # 저장 전략
    save_steps=50,                       # 저장 주기
    bf16=True,                           # bfloat16 사용
    learning_rate=1e-4,                  # 학습률
    max_grad_norm=0.3,                   # 그래디언트 클리핑
    warmup_ratio=0.03,                   # 워밍업 비율
    lr_scheduler_type="constant",        # 고정 학습률
    push_to_hub=False,                   # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [18]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

In [20]:
# 데이터의 최대 길이 한도를 지정. 최대 8192개의 토큰까지만 사용한다.
max_seq_length=8192

example = train_dataset[0]
batch = collate_fn([example])

print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 138520, 19391, 143604, 129264, 130650, 382, 13146, 48431, 20401, 66790, 29326, 131193, 17877, 125686, 125548, 139713, 624, 16, 13, 138520, 53680, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 16186, 139713, 624, 17, 13, 85322, 77226, 98801, 19391, 130768, 130213, 17877, 143604, 16186, 125476, 34395, 53900, 21329, 95577, 139713, 624, 18, 13, 138520, 19391, 128605, 143603, 12802, 85322, 77226, 98801, 19391, 130671, 32290, 85322, 77226, 98801, 126377, 330, 33883, 64795, 138520, 93, 19391, 128605, 130213, 12802, 136673, 1189, 129254, 143604, 16186, 139713, 624, 19, 13, 143604, 47836, 53618, 142976, 139236, 18411, 142616, 82190, 53435, 40853, 129549, 53435, 125068, 17877, 140174, 128836, 32290, 5140, 240, 97, 19391, 36330, 250, 125746, 16560, 23084, 126402, 83634, 17380, 94613, 139236, 84621, 47324, 18411, 129624, 20487, 139713, 13, 95617, 18411, 129901, 142976, 53435, 40

In [21]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

# 학습 시작
trainer.train()  # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()  # 최종 모델을 저장

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.553900
20,0.494500
30,0.564800
40,0.451600
50,0.456700
60,0.465400
70,0.407200
80,0.461800
90,0.399800
100,0.395800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

In [22]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [47]:
print(prompt_lst[1000])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다."라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: ‘가치’를 통해 분석한 경제 현상◇밈노믹스=‘밈(meme)’은 유전을 통하지 않고 사회 속에서 세대 간 옮겨지는 문화 요소를 뜻하는 개념이다. 저자는 밈을 경제학에 접목해 ‘시장’ 원리가 아닌 인간이 추구하는 ‘가치’의 프리즘을 통해 경제 현상을 분석하고 미래 경제를 예측한다. 혼돈과 질서의 소용돌이 속에서 변화하고 발전하는 경제 흐름을 인류의 문화적 진화라는 독특한 관점에서 해석했다. (사이드 돌리바니 지음, 박세연 옮김, 엘도라도, 444쪽, 1만8000원)唐詩 찾아 중국 삼만리◇중국, 당시의 나라=김준연 고려대 중어중문학과 교수가 이백 두보 백거이 왕유 등이 지은 당시(唐詩)의 흥취를 느끼기 위해 당나라 시대 지도를 들고 길을 나섰다. 13개 성(省)에 산재한 수십 개의 시와 현을 찾아다니며 당시 200여수의 내력을 훑었다. 10여년간 1만2500㎞를 누빈 대장정의 기록이다. 답사한 명승고적에서 직접 확인할 수 있는 시를 우선적으로 소개했다. (김준연 지음, 궁리, 652쪽, 2만8000원)클래식~팝 … 음악의 본질◇리슨 투 디스=미국 음악 비평계를 선도적으로 이끌고 있는 저자가 클

In [48]:
print(label_lst[1000])


한국 경제는 지난 수십 년간 눈부신 성장을 이루어냈지만, 이와 동시에 분배 문제도 심각하게 대두되고 있습니다. 

한국은 1960년대 이후 경제개발 5개년 계획을 통해 급격한 경제 성장을 이루었으며, 이를 통해 '한강의 기적'이라 불리는 경제적 도약을 경험했습니다. 1962년 한국의 1인당 국내총생산(GDP)은 90달러에 불과했으나, 2014년에는 2만7963달러로 311배 증가했습니다. 이러한 고도성장은 한국이 빈곤과 질병으로부터 벗어나는 데 큰 기여를 했습니다 [[ref5]].

그러나 경제 성장률이 둔화되면서 분배 문제도 심각해졌습니다. 1987년 노동자 대투쟁 이후 고임금-저생산 구조가 고착화되면서 한국 기업들이 해외로 이전하기 시작했고, 이는 경제 성장률의 둔화로 이어졌습니다. 1963년부터 1991년까지 연평균 9.5%였던 경제 성장률은 1992년부터 2011년까지 5.1%로 하락했습니다. 최근에는 2%대 성장률을 기록할 것으로 전망되고 있습니다 [[ref5]].

경제 성장률의 둔화는 소득 분배의 불평등을 악화시켰습니다. 경제 성장률이 1% 높아질 때 소득 분배의 불평등도를 나타내는 지니계수는 0.3% 감소하는 것으로 나타났지만, 성장률 침체로 인해 지니계수가 높아지고 중산층이 몰락하는 등 분배 구조가 악화되었습니다. 1990년 0.266이었던 지니계수는 2014년 0.308로 악화되었습니다 [[ref5]].

따라서 한국 경제가 다시 성장하기 위해서는 구조개혁과 규제 혁파로 투자를 활성화하고, 고부가가치 서비스 산업을 육성하는 등 성장 동력을 확충하는 것이 필요합니다. 무엇보다 중요한 것은 다시 한번 경제적 도약을 이루겠다는 의지입니다 [[ref5]].<|im_end|>



In [49]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

base_model_id = "Qwen/Qwen2-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

prompt=prompt_lst[1000]
label=label_lst[1000]
pred=test_inference(pipe, prompt)

print(f"모델의 예측:\n{test_inference(pipe, prompt)}")
print(f"정답:\n{label}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

모델의 예측:
한국 경제의 성장과 분배 문제에 대한 설명을 제공하기 위해 여러 문서를 참조하였습니다.

한국은 1960년대 이후 경제성장을 통해 '위대한 탈출'을 이뤄냈습니다. 이는 인류가 산업혁명 이후 250여년간의 비약적인 경제성장으로 소득 수준이 높아지고 기대수명이 길어져 삶의 질이 크게 개선되는 것을 의미합니다. 1962년 한국의 1인당 국내총생산(GDP)은 90달러로 미얀마의 45% 수준이었지만, 2019년에는 2만7963달러로 52년 동안 311배 증가했습니다.

그러나 성장률이 둔화되면서 분배도 악화되었습니다. 1987년 노동자 대투쟁 이후 고임금-저생산이 고착화되어 한국 기업이 해외로 탈출하기 시작했습니다. 이로 인해 한국은 1992년을 기점으로 '중성장 시대'를 맞게 되었습니다. 1963~1991년 연평균 9.5%였던 한국의 경제성장률은 1992~2011년 5.1%로 하락했습니다. 2022년의 성장률은 2%대를 기록할 것으로 전망됩니다.

이렇게 경제성장률이 떨어지면서 소득 분배의 불평등도가 높아졌습니다. 1990년 0.266이었던 지니계수는 2019년 0.308로 악화되었습니다. 

이러한 문제를 해결하기 위해서는 구조개혁과 규제혁파로 투자를 활성화하고 고부가가치 서비스산업을 육성하는 등 성장동력 확충에 최선을 다해야 합니다. 무엇보다 중요한 것은 다시 한번 일어서겠다는 대탈출에 대한 의지가 필요하다는 것이 한국 경제에 대한 분석입니다.
정답:

한국 경제는 지난 수십 년간 눈부신 성장을 이루어냈지만, 이와 동시에 분배 문제도 심각하게 대두되고 있습니다. 

한국은 1960년대 이후 경제개발 5개년 계획을 통해 급격한 경제 성장을 이루었으며, 이를 통해 '한강의 기적'이라 불리는 경제적 도약을 경험했습니다. 1962년 한국의 1인당 국내총생산(GDP)은 90달러에 불과했으나, 2014년에는 2만7963달러로 311배 증가했습니다. 이러한 고도성장은 한국이 빈곤과 질병으로부터 벗어나는 데 큰 기여를 했습니다 [[ref5]].

그러나 경제 성장

In [50]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

prompt=prompt_lst[1000]
label=label_lst[1000]
pred=test_inference(pipe, prompt)

print(f"모델의 예측:\n{test_inference(pipe, prompt)}")
print(f"정답:\n{label}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

모델의 예측:
한국 경제의 성장과 분배 문제는 여러 가지 요인으로 인해 발생하고 있습니다. 한국은 1960년대 이후 경제성장을 통해 "위대한 탈출"을 이뤄냈습니다. 이는 인류가 산업혁명 후 250여년간 비약적인 경제성장으로 소득 수준이 높아지고 기대수명이 길어져 삶의 질이 크게 개선된 것을 의미합니다[[ref5]].

그러나 이러한 성장은 분배 문제를 야기했습니다. 1987년 노동자 대투쟁 이후, 고임금-저생산이 고착화되면서 한국 기업들은 해외로 탈출하기 시작했습니다. 이로 인해 한국은 1992년을 기점으로 "중성장 시대"를 맞게 되었으며, 경제성장률이 하락했습니다. 1963~1991년 연평균 9.5%였던 한국의 경제성장률은 1992~2011년 5.1%로 하락했습니다. 2012년 이후에도 성장률은 2%대를 기록하고 있습니다[[ref5]].

경제성장률이 둔화됨에 따라 분배도 악화되었습니다. 1990년 0.266이었던 지니계수는 2012년 0.308로 악화되었습니다. 이는 경제성장률이 높아질 때 소득 분배의 불평등도를 나타내는 지니계수는 0.3% 감소하는 것으로 나타났음을 의미합니다[[ref5]].

따라서 한국 경제의 성장과 분배 문제를 해결하기 위해서는 구조개혁과 규제혁파로 투자를 활성화하고 고부가가치 서비스산업을 육성하는 등 성장동력 확충에 최선을 다해야 합니다. 또한, 다시 한번 일어서겠다는 대탈출에 대한 의지가 필요합니다[[ref5]].
정답:

한국 경제는 지난 수십 년간 눈부신 성장을 이루어냈지만, 이와 동시에 분배 문제도 심각하게 대두되고 있습니다. 

한국은 1960년대 이후 경제개발 5개년 계획을 통해 급격한 경제 성장을 이루었으며, 이를 통해 '한강의 기적'이라 불리는 경제적 도약을 경험했습니다. 1962년 한국의 1인당 국내총생산(GDP)은 90달러에 불과했으나, 2014년에는 2만7963달러로 311배 증가했습니다. 이러한 고도성장은 한국이 빈곤과 질병으로부터 벗어나는 데 큰 기여를 했습니다 [[ref5]].

그러나 경제 성장률이 둔화되